# Multi object relation T5 tokenization + T5 embedding

In [1]:
import numpy as np
import torch as th
import torch.nn as nn
import torch
from os.path import join
from tqdm.notebook import tqdm, trange
from transformers import T5Tokenizer, T5EncoderModel

/n/home12/hjkim/.conda/envs/torch2/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
dataset_root = "/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/PixArt/training_datasets/objectRelMulti_pilot1"
caption_dir = join(dataset_root, "captions")
image_dir = join(dataset_root, "images")
img_feat_dir = join(dataset_root, "img_vae_features_128resolution")
text_feat_dir = join(dataset_root, "caption_feature_wmask")

In [3]:
T5_path = "/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/PixArt/output/pretrained_models/t5_ckpts/t5-v1_1-xxl"
tokenizer = T5Tokenizer.from_pretrained(T5_path, )#subfolder="tokenizer")
encoder = T5EncoderModel.from_pretrained(T5_path)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/n/home12/hjkim/.conda/envs/torch2/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
model_max_length = 20
# use T5 tokenizer 
corpus = []
input_ids_col = []
attention_mask_col = []
for i in trange(10000):
    text = open(join(caption_dir, f"{i}.txt")).read()
    text_tokens_and_mask = tokenizer(
        text,
        max_length=model_max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )
    input_ids_col.append(text_tokens_and_mask['input_ids'])
    attention_mask_col.append(text_tokens_and_mask['attention_mask'])
    # break
    # corpus.append(text)

input_ids_tsr = th.cat(input_ids_col, dim=0)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [5]:
class T5EmbeddingEncoder(nn.Module):
    def __init__(self, model_name="t5-base", device="cuda", ):
        super().__init__()
        self.tokenizer = T5Tokenizer.from_pretrained(model_name)
        self.encoder = T5EncoderModel.from_pretrained(model_name, ).to(torch.bfloat16)
        self.encoder.eval()
        self.device = device
        self.encoder.to(device)

    def __call__(self, input_ids, attention_mask=None):
        return self.encode(input_ids, attention_mask)

    def encode(self, input_ids, attention_mask=None):
        if isinstance(input_ids, list) and isinstance(input_ids[0], str):
            # assume input_ids is raw text prompts
            tokens = self.tokenizer(input_ids, return_tensors="pt", padding=True, truncation=True)
            input_ids = tokens["input_ids"].to(self.device)
            attention_mask = tokens["attention_mask"].to(self.device)
        else:
            input_ids = input_ids.to(self.device)
            if attention_mask is None:
                attention_mask = (input_ids != self.tokenizer.pad_token_id).long().to(self.device)
            else:
                attention_mask = attention_mask.to(self.device)

        with torch.no_grad():
            outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state  # [batch_size, seq_len, hidden_dim]
        return embeddings, attention_mask

    def to(self, device):
        self.device = device
        self.encoder.to(device)
        return self

text_encoder = T5EmbeddingEncoder().to("cuda")
text_emb =  text_encoder(input_ids_tsr[0:1])[0]

In [6]:
text_emb

tensor([[[ 0.1318, -0.1138, -0.1475,  ..., -0.3457, -0.0708, -0.2520],
         [ 0.0664,  0.2441,  0.2969,  ..., -0.0522,  0.0332,  0.1709],
         [-0.2891, -0.1172, -0.2227,  ..., -0.1455, -0.0087,  0.2559],
         ...,
         [-0.0332, -0.3555, -0.2578,  ..., -0.0879,  0.1826, -0.1855],
         [-0.0559, -0.3301, -0.2715,  ..., -0.0781,  0.1885, -0.1641],
         [-0.0757, -0.3320, -0.2910,  ..., -0.0845,  0.1797, -0.1777]]],
       device='cuda:0', dtype=torch.bfloat16)

In [7]:
import sys
sys.path.append("/n/home12/hjkim/Github/DiffusionObjectRelation/PixArt-alpha")
from diffusion.model.t5 import T5Embedder


pretrain_path = "/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/PixArt/output/pretrained_models/"
t5 = T5Embedder(device="cuda", local_cache=True, cache_dir=f'{pretrain_path}/t5_ckpts', model_max_length=model_max_length)

/n/home12/hjkim/.conda/envs/torch2/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/n/home12/hjkim/.conda/envs/torch2/lib/python3.10/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/PixArt/output/pretrained_models//t5_ckpts/t5-v1_1-xxl


/n/home12/hjkim/.conda/envs/torch2/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
caption = ["triangle is above red circle"]
caption_emb, emb_mask = t5.get_text_embeddings(caption)

In [9]:
import os 

validation_prompts = [
    "triangle",
    "square",
    "circle",
    "red",
    "blue",
    "red square",
    "blue circle",
    "blue triangle",
    "a red square",
    "a blue circle",
    "a blue triangle",
    "the blue square",
    "the red circle",
    "the triangle",
    "the square",
    "the circle",
    "the",
    "or",
    "an",
    "red",
    "blue",
    "the red square above the blue circle",
    "blue triangle to the left of red square",
    "red circle below blue triangle",
    "red circle to the right of blue triangle",
    ]
max_length = 20
result_col = []
recompute = True 
device = "cuda"
prompt_cache_dir = "/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/PixArt/output/prompt_cache_t5emb_multi"
os.makedirs(prompt_cache_dir, exist_ok=True)

pretrain_path = "/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/PixArt/output/pretrained_models/"
t5 = T5Embedder(device="cuda", local_cache=True, cache_dir=f'{pretrain_path}/t5_ckpts', model_max_length=max_length)

# Save unconditioned embedding
uncond_prompt_embeds, uncond_attention_mask = t5.get_text_embeddings([""], )
torch.save({'caption_embeddings': uncond_prompt_embeds, 'emb_mask': uncond_attention_mask, 'prompt': ''}, 
            join(prompt_cache_dir,f'uncond_{max_length}token.pth'))
result_col.append({'prompt': '', 'caption_embeddingss': uncond_prompt_embeds, 'emb_mask': uncond_attention_mask})

print("Preparing Visualization prompt embeddings...")
print(f"Saving visualizate prompt text embedding at {prompt_cache_dir}")

for prompt in validation_prompts:
    if os.path.exists(join(prompt_cache_dir,f'{prompt}_{max_length}token.pth')) and not recompute:
        result_col.append(torch.load(join(prompt_cache_dir,f'{prompt}_{max_length}token.pth')))
        continue
    print(f"Mapping {prompt}...")
    caption_emb, caption_token_attention_mask =  t5.get_text_embeddings([prompt], )
    torch.save({'caption_embeddings': caption_emb, 'emb_mask': caption_token_attention_mask, 'prompt': prompt}, 
                join(prompt_cache_dir,f'{prompt}_{max_length}token.pth'))
    result_col.append({'prompt': prompt, 'caption_embeddings': caption_emb, 'emb_mask': caption_token_attention_mask})
print("Done!")

/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/PixArt/output/pretrained_models//t5_ckpts/t5-v1_1-xxl


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Preparing Visualization prompt embeddings...
Saving visualizate prompt text embedding at /n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/PixArt/output/prompt_cache_t5emb_multi
Mapping triangle...
Mapping square...
Mapping circle...
Mapping red...
Mapping blue...
Mapping red square...
Mapping blue circle...
Mapping blue triangle...
Mapping a red square...
Mapping a blue circle...
Mapping a blue triangle...
Mapping the blue square...
Mapping the red circle...
Mapping the triangle...
Mapping the square...
Mapping the circle...
Mapping the...
Mapping or...
Mapping an...
Mapping red...
Mapping blue...
Mapping the red square above the blue circle...
Mapping blue triangle to the left of red square...
Mapping red circle below blue triangle...
Mapping red circle to the right of blue triangle...
Done!


In [10]:
emb_mask.shape

torch.Size([1, 20])

In [11]:
caption_emb.shape

torch.Size([1, 20, 4096])

In [12]:
caption_emb

tensor([[[-0.0649, -0.0317,  0.0339,  ...,  0.1157, -0.0608, -0.0317],
         [-0.0289, -0.0293,  0.0474,  ...,  0.1162, -0.0918,  0.0625],
         [ 0.1064, -0.0310, -0.1289,  ...,  0.1807, -0.1094,  0.1260],
         ...,
         [ 0.1108,  0.1309, -0.0014,  ..., -0.0923, -0.0028,  0.1011],
         [ 0.0062, -0.0520,  0.0166,  ..., -0.0708,  0.0168,  0.1514],
         [-0.0732,  0.1602,  0.0664,  ...,  0.0139,  0.0193,  0.0962]]],
       device='cuda:0', dtype=torch.bfloat16)

In [74]:
th.allclose(caption_emb, embedding[ 'caption_embeds'], atol=1E-1, rtol=1E-1)

True

In [68]:
emb_mask

tensor([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       device='cuda:0')

In [67]:
embedding

{'prompt': 'triangle is above red circle',
 'caption_embeds': tensor([[[ 0.0386,  0.0007,  0.0167,  ...,  0.2578, -0.0903,  0.0042],
          [ 0.0723,  0.0742,  0.0349,  ...,  0.0586, -0.1196, -0.0781],
          [ 0.0859, -0.0747, -0.1338,  ...,  0.2041, -0.2061,  0.1104],
          ...,
          [-0.1069,  0.2070, -0.0820,  ..., -0.0559,  0.0347,  0.0674],
          [-0.1299,  0.1836, -0.1099,  ..., -0.0120,  0.0574, -0.0200],
          [-0.0781,  0.2246, -0.1523,  ..., -0.0378,  0.0069, -0.0378]]],
        device='cuda:0', dtype=torch.bfloat16),
 'emb_mask': tensor([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
        device='cuda:0')}

In [36]:
embedding["emb_mask"].shape

torch.Size([1, 20])